### Imports
---

In [1]:
import pandas as pd
import networkx as nx
from pyvis import network

#utils
from src.utils.dataset import get_network_analysis_dataset

### Dataframe preparation
---

In [2]:
transactions_df = get_network_analysis_dataset()

In [3]:
transactions_df.head(2)

,sender,receiver,value,title
0,10057_803A115E0,29467_803E020C0,10279215.10,from: 10057_803A115E0\nto: 29467_803E020C0\nam...
1,10057_803AA8E90,10099_804672160,5324508.42,from: 10057_803AA8E90\nto: 10099_804672160\nam...


In [4]:
transactions_df.shape

(100000, 4)

### Network building
---

In [5]:
relationship_network = nx.from_pandas_edgelist(
    transactions_df, 
    source="sender", 
    target="receiver", 
    edge_attr=["title","value"])

In [6]:
degree_dict_G = dict(relationship_network.degree)
nx.set_node_attributes(relationship_network, degree_dict_G, "value")

In [7]:
account_id = '10057_803AA8E90'
max_depth = 6
max_nodes = 100
physics = True

In [8]:
title_dict_G, color_dict_G  = {}, {}
for node in list(relationship_network.nodes):
    title_dict_G[node] = f"id: {node}"
    if node == account_id:
        color_dict_G[node] = "#6C757D"
    else:
        color_dict_G[node] = "#007BFF"
nx.set_node_attributes(relationship_network, title_dict_G, "title")
nx.set_node_attributes(relationship_network, color_dict_G, "color")


In [9]:
depths = nx.single_source_shortest_path_length(relationship_network, source=account_id, cutoff=max_depth)
nodes = set()
for i, node in enumerate(depths.keys()):
    if i >= max_nodes:
        break
    nodes.add(node)
subgraph = relationship_network.subgraph(nodes)

In [11]:
nt = network.Network(directed=True)
nt.from_nx(subgraph)
nt.toggle_physics(physics)
nt.save_graph('pyvis_network.html')